In [1]:
# !pip install tensorflow keras opencv-python

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import cv2
import numpy as np
import os
from tensorflow.keras.layers import BatchNormalization

In [3]:
# Skapa en ImageDataGenerator med augmentering för träningsdata
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Skapa en ImageDataGenerator för valideringsdata utan augmentering
validation_datagen = ImageDataGenerator(rescale=1./255)

# Ladda träningsdata med augmentering
train_data = train_datagen.flow_from_directory(
    'data/emotion/train',
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical'
)

# Ladda valideringsdata utan augmentering
validation_data = validation_datagen.flow_from_directory(
    'data/emotion/validation',
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical'
)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [ ]:
# Skapa och kompilera den första modellen
model = Sequential([
    Conv2D(128, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(512, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(1024, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(7, activation='softmax')
])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Skapa en Early Stopping-callback
early_stopper = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Träna modellen med Early Stopping
history = model.fit(train_data, 
                    epochs=100, 
                    validation_data=validation_data, 
                    callbacks=[early_stopper])

# Spara den första modellen
model.save('improved_emotion_detection_model_test.keras')


C:\Users\pauls\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100


C:\Users\pauls\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


901/901 ━━━━━━━━━━━━━━━━━━━━ 386s 421ms/step - accuracy: 0.1809 - loss: 2.7973 - val_accuracy: 0.3162 - val_loss: 1.7698
Epoch 2/100
901/901 ━━━━━━━━━━━━━━━━━━━━ 382s 423ms/step - accuracy: 0.2364 - loss: 2.3031 - val_accuracy: 0.3742 - val_loss: 1.6669
Epoch 3/100
901/901 ━━━━━━━━━━━━━━━━━━━━ 383s 425ms/step - accuracy: 0.2982 - loss: 2.0378 - val_accuracy: 0.4117 - val_loss: 1.5847
Epoch 4/100
901/901 ━━━━━━━━━━━━━━━━━━━━ 382s 424ms/step - accuracy: 0.3425 - loss: 1.8765 - val_accuracy: 0.4410 - val_loss: 1.5543
Epoch 5/100
901/901 ━━━━━━━━━━━━━━━━━━━━ 384s 426ms/step - accuracy: 0.3861 - loss: 1.7141 - val_accuracy: 0.4819 - val_loss: 1.4097
Epoch 6/100
901/901 ━━━━━━━━━━━━━━━━━━━━ 384s 426ms/step - accuracy: 0.4180 - loss: 1.5794 - val_accuracy: 0.5071 - val_loss: 1.3152
Epoch 7/100
901/901 ━━━━━━━━━━━━━━━━━━━━ 383s 425ms/step - accuracy: 0.4378 - loss: 1.5050 - val_accuracy: 0.5276 - val_loss: 1.2598
Epoch 8/100
901/901 ━━━━━━━━━━━━━━━━━━━━ 384s 425ms/step - accuracy: 0.4675 - los

In [ ]:
# Visualisera tränings- och valideringsförlust samt noggrannhet
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Skriv ut en sammanfattning av den alternativa modellen
alternative_model.summary()

# Visualisera tränings- och valideringsresultaten för den alternativa modellen
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(alternative_history.history['loss'], label='Training Loss')
plt.plot(alternative_history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(alternative_history.history['accuracy'], label='Training Accuracy')
plt.plot(alternative_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Hämta prestandavärden för den första modellen
new_model = tf.keras.models.load_model('improved_emotion_detection_model_test.keras')
new_train_loss, new_train_accuracy = new_model.evaluate(train_data)
new_val_loss, new_val_accuracy = new_model.evaluate(validation_data)

# Hämta prestandavärden för den andra modellen
alt_train_loss, alt_train_accuracy = alternative_model.evaluate(train_data)
alt_val_loss, alt_val_accuracy = alternative_model.evaluate(validation_data)

# Skriv ut jämförelsen mellan de två modellerna
print("=== Comparison of Models ===")
print(f"First Model - Training Accuracy: {new_train_accuracy:.4f}, Training Loss: {new_train_loss:.4f}")
print(f"First Model - Validation Accuracy: {new_val_accuracy:.4f}, Validation Loss: {new_val_loss:.4f}")
print("----------------------------------------------------")
print(f"Second Model - Training Accuracy: {alt_train_accuracy:.4f}, Training Loss: {alt_train_loss:.4f}")
print(f"Second Model - Validation Accuracy: {alt_val_accuracy:.4f}, Validation Loss: {alt_val_loss:.4f}")